# Assyrian Cuneiform Translation Baseline - VersaTech Solution Ltd

**Competition**: Translate ancient Assyrian business records to English  
**Approach**: Multilingual seq2seq with mT5 (Transfer learning from modern translation tasks)  
**Author**: Umar Abubakar ([oumar-code](https://github.com/oumar-code)) | VersaTech Solution Ltd  
**Contact**: lumarabubakarb2018@gmail.com | +234 9038650851

---

## About This Notebook
This baseline demonstrates:
- Sequence-to-sequence translation using **mT5** (multilingual T5)
- Transfer learning from modern language pairs to low-resource ancient text
- Evaluation with BLEU/chrF metrics
- Scalable NLP pipeline patterns (used in our [Akulearn EdTech platform](https://github.com/oumar-code/Akulearn_docs))

## Portfolio Context
This notebook showcases capabilities from:
- **Akulearn**: AI content generation (42 lessons, 36% WAEC coverage) → seq2seq for curriculum creation
- **African Voice Dataset**: Low-resource NLP (Hausa ASR) → transfer learning expertise
- **VersaTech Services**: Multilingual NLP, automation pipelines, ML engineering

📅 **Schedule a Consultation**: [Add Calendly link]  
📧 **Business Inquiries**: lumarabubakarb2018@gmail.com

## 1. Import Libraries & Setup

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from pathlib import Path

# NLP & Transformers
from transformers import (
    MT5ForConditionalGeneration, 
    MT5Tokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset, load_metric
import torch

# Evaluation
from sacrebleu.metrics import BLEU, CHRF

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

## 2. Load Competition Data

**Dataset**: 8,000 cuneiform texts (Old Assyrian → English)  
**Challenge**: Low-resource language, ancient script, business domain-specific vocabulary

*Note: This mirrors our work on low-resource Hausa ASR (70M speakers, <1% of English data)*

In [ ]:
# Load training data (adjust path to your Kaggle input)
DATA_PATH = Path("/kaggle/input/assyrian-translation")  # Update for local testing
train_df = pd.read_csv(DATA_PATH / "train.csv")
test_df = pd.read_csv(DATA_PATH / "test.csv")

print(f"Training samples: {len(train_df):,}")
print(f"Test samples: {len(test_df):,}")
print(f"\nColumns: {train_df.columns.tolist()}")
print(f"\nSample translation pair:")
print(f"Source (Assyrian): {train_df['source'].iloc[0]}")
print(f"Target (English): {train_df['target'].iloc[0]}")

## 3. Exploratory Data Analysis

In [ ]:
# Text length distribution
train_df['source_len'] = train_df['source'].str.len()
train_df['target_len'] = train_df['target'].str.len()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(train_df['source_len'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_title('Source Text Length Distribution')
axes[0].set_xlabel('Character Count')
axes[0].set_ylabel('Frequency')

axes[1].hist(train_df['target_len'], bins=50, color='coral', alpha=0.7, edgecolor='black')
axes[1].set_title('Target Text Length Distribution')
axes[1].set_xlabel('Character Count')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"Source Length - Mean: {train_df['source_len'].mean():.1f}, Median: {train_df['source_len'].median():.1f}")
print(f"Target Length - Mean: {train_df['target_len'].mean():.1f}, Median: {train_df['target_len'].median():.1f}")

## 4. Data Preprocessing & Tokenization

In [ ]:
# Load mT5 tokenizer
MODEL_NAME = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME)

# Preprocessing function
def preprocess_function(examples):
    """Tokenize source and target texts for seq2seq training"""
    inputs = [f"translate Assyrian to English: {text}" for text in examples['source']]
    targets = examples['target']
    
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Convert to HuggingFace Dataset
from datasets import Dataset as HFDataset
train_dataset = HFDataset.from_pandas(train_df[['source', 'target']])
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['source', 'target'])

print(f"Tokenized dataset size: {len(train_dataset):,}")
print(f"Sample tokenized input shape: {len(train_dataset[0]['input_ids'])}")

## 5. Model Training

Training mT5-small with standard seq2seq approach. For production deployment (like Akulearn's content pipeline), this would integrate with MLflow tracking and CI/CD workflows.

In [ ]:
# Load model
model = MT5ForConditionalGeneration.from_pretrained(MODEL_NAME)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # No validation split for baseline
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_steps=100,
    fp16=torch.cuda.is_available(),  # Mixed precision if GPU available
    report_to="none"
)

# Data collator for seq2seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Train
print("Starting training...")
trainer.train()
print("Training complete!")

## 6. Generate Predictions & Create Submission

In [ ]:
# Generate translations for test set
def generate_translation(text):
    """Generate English translation from Assyrian source"""
    input_text = f"translate Assyrian to English: {text}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).input_ids
    
    if torch.cuda.is_available():
        input_ids = input_ids.to('cuda')
        model.to('cuda')
    
    outputs = model.generate(
        input_ids,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate predictions
print("Generating translations for test set...")
test_df['translation'] = test_df['source'].apply(generate_translation)

# Show samples
print("\nSample translations:")
for i in range(min(3, len(test_df))):
    print(f"\nSource: {test_df['source'].iloc[i]}")
    print(f"Translation: {test_df['translation'].iloc[i]}")

# Create submission file
submission = test_df[['id', 'translation']]
submission.to_csv('submission.csv', index=False)
print(f"\nSubmission file created: {len(submission):,} translations")

## 7. Next Steps & Production Considerations

**For Production Deployment** (as implemented in [Akulearn Platform](https://github.com/oumar-code/Akulearn_docs)):
- **MLflow Tracking**: Log model metrics, hyperparameters, and artifacts
- **CI/CD Integration**: GitHub Actions for automated retraining pipelines
- **API Deployment**: FastAPI endpoint for translation service (40+ endpoints in Akulearn)
- **Monitoring**: Track inference latency, BLEU scores, user feedback

**Model Improvements**:
- Fine-tune larger mT5 variants (base, large) with extended training
- Implement back-translation for data augmentation
- Add beam search tuning and length penalties
- Ensemble multiple checkpoints

**VersaTech Services** can help with:
- Multilingual NLP pipelines for low-resource languages
- Production deployment infrastructure (Docker, Kubernetes)
- Voice integration (ASR/TTS) for accessibility
- Localized content generation at scale

📧 **Contact**: umarabubakar1960@gmail.com | 📞 +234 814 495 5037  
📅 **Schedule Consultation**: [Add Calendly link here]  
🏢 **VersaTech**: Custom AI solutions for education, healthcare, and public sector